# Cost Model Development

This notebook develops a cost optimization model for packaging quality decisions. It helps predict and minimize costs associated with packaging defects, supplier changes, and quality improvements.

## Libraries and Setup

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
import joblib
import warnings
warnings.filterwarnings('ignore')

## 1. Cost Components Definition

Define the various cost factors that impact packaging quality decisions.

In [ ]:
# Cost parameters (these would typically come from business data)
COST_PARAMETERS = {
    'defect_cost_per_unit': 5.0,  # Cost per defective package
    'rework_cost_per_unit': 2.0,  # Cost to rework/fix packaging
    'supplier_switch_cost': 10000,  # One-time cost to switch suppliers
    'quality_inspection_cost': 0.5,  # Cost per unit for quality inspection
    'premium_material_cost': 1.5,  # Additional cost for premium materials
    'customer_complaint_cost': 50,  # Average cost per customer complaint
    'brand_damage_cost': 100  # Estimated cost of brand damage per incident
}

print("Cost Parameters Defined:")
for key, value in COST_PARAMETERS.items():
    print(f"  {key}: ${value}")

## 2. Cost Calculation Functions

Functions to calculate different types of costs based on packaging quality predictions.

In [ ]:
def calculate_defect_costs(defect_probability, volume, cost_params=COST_PARAMETERS):
    """
    Calculate expected costs from packaging defects.
    
    Args:
        defect_probability: Probability of defects (0-1)
        volume: Number of units
        cost_params: Dictionary of cost parameters
    
    Returns:
        Expected defect costs
    """
    expected_defects = defect_probability * volume
    
    # Direct defect costs
    direct_costs = expected_defects * cost_params['defect_cost_per_unit']
    
    # Rework costs (assume 70% of defects can be reworked)
    rework_costs = expected_defects * 0.7 * cost_params['rework_cost_per_unit']
    
    # Customer complaint costs (assume 20% of defects lead to complaints)
    complaint_costs = expected_defects * 0.2 * cost_params['customer_complaint_cost']
    
    # Brand damage (for high defect rates)
    if defect_probability > 0.05:  # 5% threshold
        brand_damage = expected_defects * 0.1 * cost_params['brand_damage_cost']
    else:
        brand_damage = 0
    
    total_cost = direct_costs + rework_costs + complaint_costs + brand_damage
    
    return {
        'total_cost': total_cost,
        'direct_costs': direct_costs,
        'rework_costs': rework_costs,
        'complaint_costs': complaint_costs,
        'brand_damage': brand_damage,
        'expected_defects': expected_defects
    }

In [ ]:
def calculate_prevention_costs(quality_level, volume, cost_params=COST_PARAMETERS):
    """
    Calculate costs of quality prevention measures.
    
    Args:
        quality_level: Quality improvement level (0-1, where 1 is highest quality)
        volume: Number of units
        cost_params: Dictionary of cost parameters
    
    Returns:
        Prevention costs
    """
    # Inspection costs increase with quality level
    inspection_costs = volume * quality_level * cost_params['quality_inspection_cost']
    
    # Premium material costs for higher quality
    if quality_level > 0.7:
        material_premium = volume * (quality_level - 0.7) * cost_params['premium_material_cost']
    else:
        material_premium = 0
    
    total_prevention_cost = inspection_costs + material_premium
    
    return {
        'total_prevention_cost': total_prevention_cost,
        'inspection_costs': inspection_costs,
        'material_premium': material_premium
    }

In [ ]:
def calculate_total_cost(defect_probability, quality_level, volume, supplier_change=False):
    """
    Calculate total cost including defects, prevention, and supplier changes.
    
    Args:
        defect_probability: Probability of defects (0-1)
        quality_level: Quality improvement level (0-1)
        volume: Number of units
        supplier_change: Boolean, whether changing supplier
    
    Returns:
        Dictionary with cost breakdown
    """
    # Calculate component costs
    defect_costs = calculate_defect_costs(defect_probability, volume)
    prevention_costs = calculate_prevention_costs(quality_level, volume)
    
    # Supplier change cost
    supplier_cost = COST_PARAMETERS['supplier_switch_cost'] if supplier_change else 0
    
    # Total cost
    total_cost = (defect_costs['total_cost'] + 
                  prevention_costs['total_prevention_cost'] + 
                  supplier_cost)
    
    return {
        'total_cost': total_cost,
        'defect_costs': defect_costs,
        'prevention_costs': prevention_costs,
        'supplier_cost': supplier_cost,
        'cost_per_unit': total_cost / volume if volume > 0 else 0
    }

## 3. Cost Optimization Functions

Functions to find optimal quality levels and decisions to minimize total costs.

In [ ]:
def optimize_quality_level(base_defect_prob, volume, quality_levels=None):
    """
    Find optimal quality level that minimizes total cost.
    
    Args:
        base_defect_prob: Base defect probability without quality improvements
        volume: Number of units
        quality_levels: List of quality levels to test
    
    Returns:
        Optimal quality level and associated costs
    """
    if quality_levels is None:
        quality_levels = np.linspace(0, 1, 21)  # 0 to 1 in 0.05 increments
    
    results = []
    
    for quality_level in quality_levels:
        # Quality improvements reduce defect probability
        adjusted_defect_prob = base_defect_prob * (1 - quality_level * 0.8)  # Up to 80% reduction
        
        cost_breakdown = calculate_total_cost(
            defect_probability=adjusted_defect_prob,
            quality_level=quality_level,
            volume=volume
        )
        
        results.append({
            'quality_level': quality_level,
            'adjusted_defect_prob': adjusted_defect_prob,
            'total_cost': cost_breakdown['total_cost'],
            'cost_per_unit': cost_breakdown['cost_per_unit'],
            'cost_breakdown': cost_breakdown
        })
    
    # Find optimal (minimum cost) solution
    optimal_result = min(results, key=lambda x: x['total_cost'])
    
    return optimal_result, results

In [ ]:
def compare_supplier_options(current_defect_prob, new_supplier_defect_prob, volume):
    """
    Compare costs of staying with current supplier vs switching.
    
    Args:
        current_defect_prob: Current supplier's defect probability
        new_supplier_defect_prob: New supplier's defect probability
        volume: Number of units
    
    Returns:
        Comparison results and recommendation
    """
    # Optimize quality for current supplier
    current_optimal, _ = optimize_quality_level(current_defect_prob, volume)
    
    # Optimize quality for new supplier (including switch cost)
    new_optimal, _ = optimize_quality_level(new_supplier_defect_prob, volume)
    new_total_cost = new_optimal['total_cost'] + COST_PARAMETERS['supplier_switch_cost']
    
    # Calculate savings
    cost_savings = current_optimal['total_cost'] - new_total_cost
    
    recommendation = {
        'switch_supplier': cost_savings > 0,
        'cost_savings': cost_savings,
        'current_supplier': {
            'total_cost': current_optimal['total_cost'],
            'optimal_quality_level': current_optimal['quality_level'],
            'final_defect_prob': current_optimal['adjusted_defect_prob']
        },
        'new_supplier': {
            'total_cost_with_switch': new_total_cost,
            'optimal_quality_level': new_optimal['quality_level'],
            'final_defect_prob': new_optimal['adjusted_defect_prob']
        }
    }
    
    return recommendation

## 4. Cost Visualization Functions

Functions to visualize cost trade-offs and optimization results.

In [ ]:
def plot_cost_optimization(base_defect_prob, volume):
    """
    Plot cost components vs quality level to visualize optimization.
    """
    optimal_result, all_results = optimize_quality_level(base_defect_prob, volume)
    
    # Extract data for plotting
    quality_levels = [r['quality_level'] for r in all_results]
    total_costs = [r['total_cost'] for r in all_results]
    defect_costs = [r['cost_breakdown']['defect_costs']['total_cost'] for r in all_results]
    prevention_costs = [r['cost_breakdown']['prevention_costs']['total_prevention_cost'] for r in all_results]
    
    # Create the plot
    plt.figure(figsize=(12, 8))
    
    plt.subplot(2, 2, 1)
    plt.plot(quality_levels, total_costs, 'b-', linewidth=2, label='Total Cost')
    plt.axvline(optimal_result['quality_level'], color='r', linestyle='--', alpha=0.7, label='Optimal')
    plt.xlabel('Quality Level')
    plt.ylabel('Total Cost ($)')
    plt.title('Total Cost vs Quality Level')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.subplot(2, 2, 2)
    plt.plot(quality_levels, defect_costs, 'r-', label='Defect Costs')
    plt.plot(quality_levels, prevention_costs, 'g-', label='Prevention Costs')
    plt.xlabel('Quality Level')
    plt.ylabel('Cost ($)')
    plt.title('Cost Components')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.subplot(2, 2, 3)
    defect_probs = [r['adjusted_defect_prob'] for r in all_results]
    plt.plot(quality_levels, defect_probs, 'orange', linewidth=2)
    plt.xlabel('Quality Level')
    plt.ylabel('Defect Probability')
    plt.title('Quality Level Effect on Defects')
    plt.grid(True, alpha=0.3)
    
    plt.subplot(2, 2, 4)
    cost_per_unit = [r['cost_per_unit'] for r in all_results]
    plt.plot(quality_levels, cost_per_unit, 'purple', linewidth=2)
    plt.axvline(optimal_result['quality_level'], color='r', linestyle='--', alpha=0.7)
    plt.xlabel('Quality Level')
    plt.ylabel('Cost per Unit ($)')
    plt.title('Unit Cost vs Quality Level')
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Print optimization results
    print(f"\nOptimization Results for Volume: {volume:,} units")
    print(f"Base Defect Probability: {base_defect_prob:.1%}")
    print(f"\nOptimal Quality Level: {optimal_result['quality_level']:.1%}")
    print(f"Resulting Defect Probability: {optimal_result['adjusted_defect_prob']:.1%}")
    print(f"Total Cost: ${optimal_result['total_cost']:,.2f}")
    print(f"Cost per Unit: ${optimal_result['cost_per_unit']:.2f}")

In [ ]:
def plot_supplier_comparison(current_defect_prob, new_supplier_defect_prob, volume):
    """
    Visualize supplier comparison results.
    """
    recommendation = compare_supplier_options(current_defect_prob, new_supplier_defect_prob, volume)
    
    # Create comparison visualization
    suppliers = ['Current\nSupplier', 'New Supplier\n(with switch cost)']
    costs = [recommendation['current_supplier']['total_cost'],
             recommendation['new_supplier']['total_cost_with_switch']]
    
    plt.figure(figsize=(12, 6))
    
    plt.subplot(1, 2, 1)
    bars = plt.bar(suppliers, costs, color=['lightblue', 'lightcoral'])
    plt.ylabel('Total Cost ($)')
    plt.title('Supplier Cost Comparison')
    
    # Add value labels on bars
    for bar, cost in zip(bars, costs):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + max(costs)*0.01, 
                f'${cost:,.0f}', ha='center', va='bottom')
    
    plt.grid(True, alpha=0.3)
    
    # Quality levels comparison
    plt.subplot(1, 2, 2)
    quality_levels = [recommendation['current_supplier']['optimal_quality_level'],
                      recommendation['new_supplier']['optimal_quality_level']]
    
    bars = plt.bar(suppliers, quality_levels, color=['lightgreen', 'lightyellow'])
    plt.ylabel('Optimal Quality Level')
    plt.title('Optimal Quality Investment')
    
    # Add value labels
    for bar, quality in zip(bars, quality_levels):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
                f'{quality:.1%}', ha='center', va='bottom')
    
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    # Print recommendation
    print(f"\nSupplier Recommendation for Volume: {volume:,} units")
    print(f"Current Supplier Defect Rate: {current_defect_prob:.1%}")
    print(f"New Supplier Defect Rate: {new_supplier_defect_prob:.1%}")
    print(f"\nRecommendation: {'SWITCH' if recommendation['switch_supplier'] else 'STAY'}")
    
    if recommendation['switch_supplier']:
        print(f"Expected Savings: ${recommendation['cost_savings']:,.2f}")
    else:
        print(f"Additional Cost if Switched: ${-recommendation['cost_savings']:,.2f}")
    
    return recommendation

## 5. Example Usage and Testing

Demonstrate the cost model with realistic scenarios.

In [ ]:
# Example 1: Quality Level Optimization
print("=== EXAMPLE 1: Quality Level Optimization ===")

# Scenario parameters
base_defect_rate = 0.08  # 8% base defect rate
production_volume = 10000  # 10,000 units

# Run optimization
plot_cost_optimization(base_defect_rate, production_volume)

In [ ]:
# Example 2: Supplier Comparison
print("\n=== EXAMPLE 2: Supplier Comparison ===")

# Scenario parameters
current_supplier_defect_rate = 0.12  # 12% defect rate
new_supplier_defect_rate = 0.06     # 6% defect rate
production_volume = 15000           # 15,000 units

# Run comparison
supplier_recommendation = plot_supplier_comparison(
    current_supplier_defect_rate, 
    new_supplier_defect_rate, 
    production_volume
)

## 6. Sensitivity Analysis

Analyze how changes in cost parameters affect optimal decisions.

In [ ]:
def sensitivity_analysis(base_scenario, parameter_variations):
    """
    Perform sensitivity analysis on cost parameters.
    
    Args:
        base_scenario: Dictionary with base scenario parameters
        parameter_variations: Dictionary of parameter variations to test
    """
    base_defect_prob = base_scenario['defect_prob']
    volume = base_scenario['volume']
    
    # Get baseline optimal result
    baseline_optimal, _ = optimize_quality_level(base_defect_prob, volume)
    
    print(f"Sensitivity Analysis Results")
    print(f"Baseline Optimal Quality Level: {baseline_optimal['quality_level']:.1%}")
    print(f"Baseline Total Cost: ${baseline_optimal['total_cost']:,.2f}")
    print("\n" + "="*60)
    
    results = {}
    
    for param_name, variations in parameter_variations.items():
        print(f"\nParameter: {param_name}")
        print(f"{'Variation':<15} {'Quality Level':<15} {'Total Cost':<15} {'Change':<10}")
        print("-" * 60)
        
        param_results = []
        
        for variation in variations:
            # Temporarily modify the cost parameter
            original_value = COST_PARAMETERS[param_name]
            COST_PARAMETERS[param_name] = original_value * variation
            
            # Run optimization with modified parameter
            optimal_result, _ = optimize_quality_level(base_defect_prob, volume)
            
            # Calculate changes
            cost_change = (optimal_result['total_cost'] - baseline_optimal['total_cost']) / baseline_optimal['total_cost'] * 100
            
            param_results.append({
                'variation': variation,
                'quality_level': optimal_result['quality_level'],
                'total_cost': optimal_result['total_cost'],
                'cost_change_pct': cost_change
            })
            
            print(f"{variation:<15.1f} {optimal_result['quality_level']:<15.1%} ${optimal_result['total_cost']:<14,.0f} {cost_change:>+6.1f}%")
            
            # Restore original parameter value
            COST_PARAMETERS[param_name] = original_value
        
        results[param_name] = param_results
    
    return results

In [ ]:
# Run sensitivity analysis
base_scenario = {
    'defect_prob': 0.10,  # 10% base defect rate
    'volume': 20000       # 20,000 units
}

# Test variations of key parameters
parameter_variations = {
    'defect_cost_per_unit': [0.5, 0.8, 1.0, 1.5, 2.0],  # 50% to 200% of base
    'quality_inspection_cost': [0.5, 0.8, 1.0, 1.5, 2.0],
    'premium_material_cost': [0.5, 0.8, 1.0, 1.5, 2.0],
    'customer_complaint_cost': [0.5, 0.8, 1.0, 1.5, 2.0]
}

sensitivity_results = sensitivity_analysis(base_scenario, parameter_variations)

## 7. Model Integration Functions

Functions to integrate the cost model with risk and recommendation models.

In [ ]:
def integrate_with_risk_model(risk_predictions, volumes, cost_params=None):
    """
    Integrate cost model with risk model predictions.
    
    Args:
        risk_predictions: Array of risk probabilities from risk model
        volumes: Array of production volumes
        cost_params: Optional custom cost parameters
    
    Returns:
        DataFrame with cost-optimized recommendations
    """
    if cost_params:
        global COST_PARAMETERS
        original_params = COST_PARAMETERS.copy()
        COST_PARAMETERS.update(cost_params)
    
    results = []
    
    for i, (risk_prob, volume) in enumerate(zip(risk_predictions, volumes)):
        # Get optimal quality level for this risk/volume combination
        optimal_result, _ = optimize_quality_level(risk_prob, volume)
        
        results.append({
            'sample_id': i,
            'predicted_risk': risk_prob,
            'volume': volume,
            'optimal_quality_level': optimal_result['quality_level'],
            'expected_defects': optimal_result['cost_breakdown']['defect_costs']['expected_defects'],
            'total_cost': optimal_result['total_cost'],
            'cost_per_unit': optimal_result['cost_per_unit'],
            'recommendation': 'HIGH_QUALITY' if optimal_result['quality_level'] > 0.7 else 'STANDARD_QUALITY'
        })
    
    # Restore original parameters if they were modified
    if cost_params:
        COST_PARAMETERS = original_params
    
    return pd.DataFrame(results)

In [ ]:
# Example integration with hypothetical risk model predictions
print("=== MODEL INTEGRATION EXAMPLE ===")

# Simulate risk model predictions
np.random.seed(42)
n_samples = 100
simulated_risk_predictions = np.random.beta(2, 8, n_samples)  # Skewed towards lower risk
simulated_volumes = np.random.randint(1000, 50000, n_samples)

# Integrate with cost model
integrated_results = integrate_with_risk_model(simulated_risk_predictions, simulated_volumes)

# Display summary statistics
print(f"\nIntegrated Results Summary ({n_samples} samples):")
print(f"Average Predicted Risk: {integrated_results['predicted_risk'].mean():.1%}")
print(f"Average Optimal Quality Level: {integrated_results['optimal_quality_level'].mean():.1%}")
print(f"Average Cost per Unit: ${integrated_results['cost_per_unit'].mean():.2f}")
print(f"\nRecommendation Distribution:")
print(integrated_results['recommendation'].value_counts())

# Show first few rows
print(f"\nFirst 10 recommendations:")
display_columns = ['predicted_risk', 'volume', 'optimal_quality_level', 'cost_per_unit', 'recommendation']
print(integrated_results[display_columns].head(10).to_string(index=False, float_format='%.3f'))

## 8. Model Saving and Deployment

Save the cost model components for deployment in production systems.

In [ ]:
# Save cost model components
import json
import os

# Create output directory
output_dir = '../data/preprocessed/model_outputs/'
os.makedirs(output_dir, exist_ok=True)

# Save cost parameters
with open(f'{output_dir}/cost_parameters.json', 'w') as f:
    json.dump(COST_PARAMETERS, f, indent=2)

# Save cost model functions as a module (for deployment)
cost_model_code = '''
# Cost Model Functions for Production Deployment
import numpy as np
import json

# Load cost parameters
with open('cost_parameters.json', 'r') as f:
    COST_PARAMETERS = json.load(f)

''' + '''\n# Add the key functions here for deployment\n'''

with open(f'{output_dir}/cost_model.py', 'w') as f:
    f.write(cost_model_code)

# Save a sample of integrated results for testing
integrated_results.to_csv(f'{output_dir}/cost_optimization_sample.csv', index=False)

print("Cost model components saved successfully!")
print(f"Output directory: {output_dir}")
print(f"Files saved:")
print(f"  - cost_parameters.json")
print(f"  - cost_model.py")
print(f"  - cost_optimization_sample.csv")